<a href="https://colab.research.google.com/github/jorgik1/ai-site-generator/blob/main/site_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai python-dotenv langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -
!sudo apt-get install -y nodejs
!npm install -g vercel


## Installing the NodeSource Node.js 16.x repo...


## Populating apt-get cache...

+ apt-get update
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,009 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-

In [7]:
import os
import subprocess

import openai
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from google.colab import files
uploaded = files.upload()
load_dotenv("./.env")
VERCEL_TOKEN = os.environ.get("VERCEL_TOKEN")
openai.api_key = os.environ.get("OPENAI_API_KEY")


vercel_website_name = "developer-portfolio"

import requests
import tempfile
from pathlib import Path
import json

def deployVercel(html_content: str, project_name: str, vercel_token: str):
    with tempfile.TemporaryDirectory() as temp_dir:
        project_dir = Path(temp_dir) / project_name
        os.makedirs(project_dir)

        # Write the HTML content to the index.html file
        with open(project_dir / "index.html", "w") as f:
            f.write(html_content)

        # Write the vercel.json configuration file
        vercel_config = {
            "name": project_name,
            "version": 2,
            "builds": [{"src": "index.html", "use": "@vercel/static"}],
        }

        with open(project_dir / "vercel.json", "w") as f:
            json.dump(vercel_config, f)

        # Deploy to Vercel using the CLI
        cmd = ["vercel", "--token", vercel_token, "-y", "--prod"]

        result = subprocess.run(cmd, cwd=project_dir, capture_output=True, text=True)

        if result.returncode == 0:
            print("Deployment READY!!")
            print(f"Final URL: {result.stdout.strip()}")
        else:
            print("Deployment failed:")
            print(result.stderr)

def create_website(github_json):
    prompt = f"""Make a beautiful website. Provide only HTML and related code. Your code should not only be visually stunning, but also intuitive and mobile friendly. Include some animations where appropriate. 
             Remember to prioritize readability and organization, and to create a polished final product. Include the following information from this json in the website, 
             but skip the ones that end in url. Do not provide any other text or answer: 
            ---
            {github_json}"""

    chat = ChatOpenAI(temperature=0.2, max_tokens=2500)

    messages = [
        SystemMessage(content="You are a AI senior frontend designer and developer trained by OpenAI. Generate HTML that is colorful and well designed. Use gradient and animations where applicable."),
        HumanMessage(content=prompt),
    ]

    response = chat(messages)

    reply = response.content
    return reply

def createPortfolio(user):
    response = requests.get(f"https://api.github.com/users/{user}")
    if response.status_code != 200:
        print(f"Error: Unable to fetch the page. Status code: {response.status_code}")
        return
    print(response.json())

    html_content = create_website(response.json())
    deployVercel(html_content, vercel_website_name, VERCEL_TOKEN)

user = input("What developer github ID to make personal website for?")

createPortfolio(user)

Saving .env to .env (7)
What developer github ID to make personal website for?jorgik1
{'login': 'jorgik1', 'id': 10710213, 'node_id': 'MDQ6VXNlcjEwNzEwMjEz', 'avatar_url': 'https://avatars.githubusercontent.com/u/10710213?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/jorgik1', 'html_url': 'https://github.com/jorgik1', 'followers_url': 'https://api.github.com/users/jorgik1/followers', 'following_url': 'https://api.github.com/users/jorgik1/following{/other_user}', 'gists_url': 'https://api.github.com/users/jorgik1/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/jorgik1/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/jorgik1/subscriptions', 'organizations_url': 'https://api.github.com/users/jorgik1/orgs', 'repos_url': 'https://api.github.com/users/jorgik1/repos', 'events_url': 'https://api.github.com/users/jorgik1/events{/privacy}', 'received_events_url': 'https://api.github.com/users/jorgik1/received_events', 'type': 'User', 'sit